In [2]:
from __future__ import print_function, division

import keras.losses
from keras.datasets import mnist
from keras import utils, callbacks
from keras.layers import Input, Dense, Reshape, Flatten,\
    Dropout, Convolution2D, MaxPooling2D, \
    AveragePooling2D, Convolution2DTranspose, Conv2DTranspose,GlobalAveragePooling2D
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.activation import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam,SGD
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
import cv2 as cv
import glob
import numpy as np

def load_preprosess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    return image

def readPicture(batch_size):
    image_path = glob.glob(r"autodl-nas/defect_image/*.png")
    img_ds = tf.data.Dataset.from_tensor_slices(image_path)
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    img_ds = img_ds.map(load_preprosess_image, num_parallel_calls=AUTOTUNE)
    BATCH_SIZE = batch_size
    image_count = len(image_path)
    img_ds = img_ds.shuffle(image_count).batch(BATCH_SIZE)
    img_ds = img_ds.prefetch(AUTOTUNE)
    return img_ds

class DCGAN(Model):
    def __init__(self):
        super().__init__()
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        self.optimizer = Adam(0.00001)

    def generator_model(self):
        model = Sequential()
        model.add(Dense(8 * 8 * 256, use_bias=False, input_shape=(100,)))
        model.add(BatchNormalization())
        model.add(LeakyReLU())

        model.add(Reshape((8, 8, 256)))  # 输出8*8*256
        # 反卷积
        model.add(Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias='false'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())  # 输出8*8*128

        model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias='false'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())  # 输出16*16*64

        model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias='false'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())  # 输出32*32*32

        model.add(Conv2DTranspose(16, (5, 5), strides=(2, 2), padding='same', use_bias='false'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())  # 输出64*64*16

        model.add(Conv2DTranspose(3, (5, 5),
                                         strides=(2, 2),
                                         padding='same',
                                         use_bias=False,
                                         activation='tanh'))  # 输出128*128*3

        return model


    def discriminator_model(self):
        model = Sequential()
        model.add(Conv2D(16,
                         (5, 5),
                         strides=(2, 2),
                         padding='same',
                         input_shape=(128, 128, 3)))  # 输入为64*64*3
        model.add(LeakyReLU())
        model.add(Dropout(0.3))  # 64*64*16

        model.add(Conv2D(32,
                                (5, 5),
                                strides=(2, 2),
                                padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.3))  # 32*32*32

        model.add(Conv2D(64,
                         (5, 5),
                         strides=(2, 2),
                         padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.3))  # 16*16*64

        model.add(Conv2D(128,
                         (5, 5),
                         strides=(2, 2),
                         padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.3))  # 8*8*128

        model.add(Conv2D(256,
                                (5, 5),
                                strides=(2, 2),
                                padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())  # 4*4*256

        model.add(GlobalAveragePooling2D())
        # Global Average Pooling(简称GAP，全局池化层)，被认为是可以替代全连接层的一种新技术

        model.add(Dense(1024))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dense(1, activation='sigmoid'))
        return model

    def d_on_g(self, d, g):
        model = Sequential()
        model.add(g)
        d.trainable = False
        model.add(d)
        d.trainable = True
        return model

    def train(self, batch_size, epochs):
        discriminator = self.discriminator_model()
        discriminator.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])
        generator = self.generator_model()
        generator.compile(loss='binary_crossentropy', optimizer=self.optimizer)
        d_on_g = self.d_on_g(discriminator, generator)
        d_on_g.compile(loss='binary_crossentropy', optimizer=self.optimizer)

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            all_imgs = readPicture(batch_size)
            ct = 1
            for batch in all_imgs:
                if batch.shape[0] != batch_size:
                    continue
                # ---------------------
                #  Train Discriminator
                # ---------------------

                # Select a random batch of images

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                gen_imgs = generator.predict(noise)

                d_loss_real = discriminator.train_on_batch(batch, valid)
                d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator (to have the discriminator label samples as valid)
                g_loss = d_on_g.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d batch: %d [D loss: %f accuracy: %.2f] [G loss: %f]" % (epoch, ct, d_loss[0], 100*d_loss[1], g_loss))
                ct += 1

            if epoch % 50 == 0:
                generator.save_weights('trainmodel/fg'+ str(epoch) + '.h5')
                discriminator.save_weights('trainmodel/fd' + str(epoch) + '.h5')
                noise = np.random.normal(0, 1, (4, self.latent_dim))
                pre_image = generator.predict(noise)
                fig = plt.figure(figsize=(16, 3))  # figsize:指定figure的宽和高，单位为英寸
                for i in range(pre_image.shape[0]):  # pre_image的shape的第一个维度就是个数，这里是6
                    plt.subplot(1, 4, i + 1)  # 几行几列的 第i+1个图片（从1开始）
                    plt.imshow((pre_image[i, :, :, :] + 1) / 2)  # 加1除2: 将生成的-1～1的图片弄到0-1之间,
                    plt.axis('off')  # 不要坐标
                plt.savefig("images/%d.png" % epoch)

2023-04-10 21:43:02.578252: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
gan = DCGAN()
gan.train(32, 700)
gan.save('dcgan_fabric.h5')

2023-04-10 21:43:04.905096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 21:43:05.551505: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8090 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:b5:00.0, compute capability: 8.6
2023-04-10 21:43:07.726076: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-10 21:43:08.475461: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


1/1 [==============================] - 3s 3s/step
epoch:0 batch: 1 [D loss: 0.812010 accuracy: 51.56] [G loss: 1.037738]
1/1 [==============================] - 0s 23ms/step
epoch:0 batch: 2 [D loss: 0.774993 accuracy: 54.69] [G loss: 0.972775]
1/1 [==============================] - 0s 32ms/step
epoch:0 batch: 3 [D loss: 0.801478 accuracy: 54.69] [G loss: 1.015659]
1/1 [==============================] - 0s 23ms/step
epoch:0 batch: 4 [D loss: 0.771072 accuracy: 45.31] [G loss: 1.007247]
1/1 [==============================] - 0s 21ms/step
epoch:0 batch: 5 [D loss: 0.779614 accuracy: 51.56] [G loss: 0.982536]
1/1 [==============================] - 0s 23ms/step
epoch:0 batch: 6 [D loss: 0.749672 accuracy: 48.44] [G loss: 0.959919]
1/1 [==============================] - 0s 20ms/step
epoch:0 batch: 7 [D loss: 0.780865 accuracy: 50.00] [G loss: 0.953076]
1/1 [==============================] - 0s 20ms/step
epoch:0 batch: 8 [D loss: 0.741730 accuracy: 50.00] [G loss: 0.980703]
1/1 [=============